In [1]:
import os
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
chat = ChatOpenAI(
    openai_api_key=openai_api_key,
    model='gpt-3.5-turbo'
)

In [3]:
from langchain.schema import (
    SystemMessage,
    HumanMessage
)

def get_user_msg(user_msg: str):
    messages = [
        SystemMessage(content='Ты умный ассистент, отвечающий на вопросы пользователя о файлах, содержащихся в базе данных. Полезные для ответа части текста из файлов будут поданы в контексте. Используй их, чтобы дать точный и полный ответ на вопрос пользователя. Не матерись. Все матерные слова заменяй на звездочки.'),
        HumanMessage(content=user_msg)
    ]
    return chat(messages).content

get_user_msg("Привет, расскажи про LLM")

'Привет! LLM - это сокращение от "Master of Laws", что в переводе с английского означает "Магистр права". LLM - это академическая степень, которую можно получить после окончания юридического образования. Программы LLM предоставляют студентам возможность углубить свои знания в определенной области права или получить более широкий обзор различных правовых дисциплин.\n\nПрограммы LLM предлагаются во многих университетах по всему миру. Они могут быть ориентированы на различные специализации, такие как корпоративное право, международное право, право интеллектуальной собственности и др. Продолжительность программы обычно составляет один год, но в некоторых случаях может быть и двухлетней.\n\nLLM-программы предлагают студентам возможность изучать различные аспекты права, а также развивать навыки анализа, исследования и письма. Эта степень может быть полезной для тех, кто хочет продолжить карьеру в юридической области, работать в правительственных организациях, юридических фирмах или междунаро

In [4]:
from langchain.document_loaders import DirectoryLoader
DATA_PATH = 'data'
def load_documents():
    loader = DirectoryLoader(DATA_PATH, glob='*.md')
    documents = loader.load()
    return documents

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
def split_text(documents: list):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=500,
        length_function=len,
        add_start_index=True
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(chunks)} chunks.")
    document = chunks[10]
    print(document.page_content)
    print(document.metadata)
    return chunks

In [6]:
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings
import shutil


CHROMA_PATH = "chroma"


def save_to_chroma(chunks: list):
    if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)
    db = Chroma.from_documents(
        chunks, OpenAIEmbeddings(), persist_directory=CHROMA_PATH
    )
    db.persist()
    print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")

In [7]:
def generate_data_store():
    documents = load_documents()
    chunks = split_text(documents)
    save_to_chroma(chunks)
generate_data_store()

Split 1 documents into 39 chunks.
Подкрадулями и бархатными тягами называли любую странную обувь. Если первое произошло от дословного перевода слова sneakers, то второе — от голосового сообщения из «Вотсапа», в котором неизвестный мужчина воодушевленно о чем-то говорит: «Что за тяги такие, бархатные тяги, ребята? Уф-ф-ф. Кефтеме».

В итоге с помощью этих слов не только иронизировали над кроссовками с острыми носами, но и хвастались собственной обувью.

Я в шоке Давай

За что мы любим мем: он поддержит нас в любых начинаниях

Формат: видео, крылатая фраза

Пик популярности: конец марта

Где расходился: «Тикток», «Инстаграм»*

Кто мог подумать, что короткий отрывок из обзора на мороженое «Баунти» так разойдется. Но тут сошлось все: и эмоциональная яркая фраза, и ее универсальность, и атмосфера одобрения. Все бы хотели такого друга, который даже в состоянии шока от идеи все равно тебя поддержит.

Папа римский в Balenciaga

За что мы любим мем: нейросети раскрыли свой мемный потенциал

Фор

In [14]:
from langchain_core.prompts import ChatPromptTemplate
import argparse
PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""
def main(query_text):

    # Prepare the DB.
    embedding_function = OpenAIEmbeddings()
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

    # Search the DB.
    results = db.similarity_search_with_relevance_scores(query_text, k=3)
    if len(results) == 0 or results[0][1] < 0.7:
        print(f"Unable to find matching results.")
        return

    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)
    print(prompt)

    model = ChatOpenAI()
    response_text = model.predict(prompt)

    sources = [doc.metadata.get("source", None) for doc, _score in results]
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    print(formatted_response)

In [15]:
main('Кто такие Сигмы?')

Human: 
Answer the question based only on the following context:

Возглас одобрения с ненормативной лексикой, который рифмуется с «могучий», прицепился к псу в рунете. Так Чиро получил популярность в России, а заодно стал продолжателем мемных традиций котенка Чмони.

Сигмы

За что мы любим мем: попытка переосмыслить маскулинность — то ли в шутку, то ли всерьез

Формат: мультиформатный

Пик популярности: зима, но вообще — весь год

Где расходился: «Тикток», «Инстаграм»*, «Ютуб»

Сигмами называют крутых парней — главных героев «Бойцовского клуба», «Острых козырьков» или «Джона Уика». И тех, кто очень хочет быть на них похожим и видит в персонажах буквально себя. Смысл термина постоянно трансформируется. Сигмы и одинокие волки, и сверхуспешные трудоголики, и образцы маскулинности, которые с уважением относятся к окружающим.

---

Формат: мультиформатный

Пик популярности: зима, но вообще — весь год

Где расходился: «Тикток», «Инстаграм»*, «Ютуб»

Сигмами называют крутых парней — главных г